# Downloading Hearthstone Data

Thanks to the great <a href="https://market.mashape.com/omgvamp/hearthstone#">OMG VAMP hearth API</a> we can download the data of all cards and structure it as we need.

For this use-case I have saved the data as a CSV, structuring it in a way that will allow me to work with it using Pandas and other Data Science python libraries.

In [10]:
import unirest as requests
import json
import csv
from collections import Counter

In [11]:
# HTTP Request for Cards Data
request_headers = {'X-Mashape-Key':'vYlGUlNJVUmshZe4dtiXYPmEwDYgp1XSB4rjsnnlqYok4ygKpT'}
response = requests.get('https://omgvamp-hearthstone-v1.p.mashape.com/cards', headers=request_headers)
all_cards = response.body

In [12]:
# Defining Dictionary Fixing Function
def fix_dict_encoding(dictionary):
    """
    'Fixes' the text of all keys of the dictionary by attempting to replace characters that are problematic
    to be encoded in unicode by their unicode representation
    """
    
    for key, value in dictionary.items():

        # Is it a string ? (Otherwise it won't need fixing)
        if isinstance(value, unicode):
            dictionary[key] = dictionary[key].replace(u'\u2019', u'\'').replace(u'\u2026',u' ').replace(u'\u2018',u'\'')
            dictionary[key] = dictionary[key].replace(u'\u201c', u'').replace(u'\u201d', u'').replace(u'\xa0',u' ')
            dictionary[key] = dictionary[key].replace(u'\xf1', u'n').replace(u'\u2122', u'').replace(u'\xe9', 'e')
            dictionary[key] = dictionary[key].replace(u'\u2013', u'').replace(u'\xf6', u'o')
            
    return dictionary

In [13]:
# Dumping data from in memory dictionary to a csv_file
with open('hearth.csv', 'wb') as f_writer:
    keys = ['cardId', 'name', 'cardSet', 'type', 'rarity', 'mechanics_list', 'mechanics_count', 'playerClass','faction', 'cost',
            'health', 'attack', 'collectible', 'race', 'artist','text', 'flavor', 'locale']
    
    # Defining DictWriter class (Including headers and setting it to ignore extra fields found on the dict)
    w = csv.DictWriter(f_writer, fieldnames=keys, extrasaction='ignore')
    w.writeheader()
    
    # Counters
    counters = Counter()
    for group, group_cards in all_cards.items():
        
        for card in group_cards:
            
            try:
                if 'mechanics' in card:
                    card['mechanics_count'] = len(card['mechanics'])                    
                    
                    # Appending elements to it's 'list' of mechanics 
                    mechanics = []
                    for item in card['mechanics']:
                        mechanics.append(item['name'])
                        
                    card['mechanics_list'] = '#'.join(mechanics)
                else:
                    card['mechanics_count'] = 0
                    card['mechanics_list'] = None
                    
                # Hard-fixing encoding characters because of encoding errors
                card = fix_dict_encoding(card)
                    
                w.writerow(card)
                counters[group] += 1
                
            except UnicodeEncodeError as ex:
                print str(card) + '\n=========================\n'
                counters['UnicodeEncodeError'] += 1

print counters.most_common()

[(u'Classic', 394), (u'The League of Explorers', 254), (u'Tavern Brawl', 248), (u'Karazhan', 228), (u'Blackrock Mountain', 221), (u'Whispers of the Old Gods', 217), (u'Basic', 209), (u'The Grand Tournament', 206), (u'Goblins vs Gnomes', 180), (u'Naxxramas', 160), (u'Debug', 85), (u'Credits', 46), (u'Missions', 36), (u'Hero Skins', 21), (u'Promo', 13), (u'Reward', 2), (u'System', 1)]
